In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
from matplotlib_scalebar.scalebar import ScaleBar
from tqdm import tqdm
import numpy as np
from datetime import datetime

In [2]:
# Establish directory locations

parent_dir    = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_dir      = os.path.join(parent_dir, 'data')
satellite_dir = os.path.join(data_dir, 'satellite')
output_dir    = os.path.join(parent_dir, 'Output')
pos_data_dir  = os.path.join(parent_dir, 'Plotting_tools/shared_data')

In [3]:
all_positions = pd.read_csv(os.path.join(pos_data_dir, 'rt_positions.csv'))
all_positions['datetime'] = pd.to_datetime(all_positions['date'])

In [4]:
lovuse31 = all_positions[all_positions['platform_id'] == 'lovuse031c']

In [5]:
platform_mask = {
        'Ship':  'x',
        'glider': '^',
        'Float':  'o',
        'respire': '>'}

platform_colors = {
        'Discovery':  'black',
        'unit_405': '#b2182b',
        'unit_397':  '#f4a582',
        'unit_398': '#d6604d',
        'unit_345': '#fddbc7',
        'lovuse031c': '#92c5de',
        'lovuse032c': '#4393c3',
        'lovuse026d': '#2166ac',
        'respire': '#b8e186'}

def get_color(platform_id):
    return platform_colors.get(platform_id, 'black')

all_positions['color'] = all_positions['platform_id'].apply(get_color)

In [6]:
autonomy_positions = all_positions[all_positions['platform_type'].isin(['glider', 'Float'])]

In [7]:
max_lon = autonomy_positions['lon'].max() + 0.05
min_lon = autonomy_positions['lon'].min() - 0.05
max_lat = autonomy_positions['lat'].max() + 0.05
min_lat = autonomy_positions['lat'].min() - 0.05


In [8]:
positions = all_positions.loc[all_positions.groupby('platform_id')['date'].idxmax()]

In [9]:

current_time = datetime.now()
now = current_time.strftime(format = '%Y-%m-%d %H:%M:%S')

In [10]:
positions_from_start = all_positions[all_positions['datetime'] > pd.to_datetime('2024-06-03 05:00:01')]

In [11]:
floats_positions = all_positions[all_positions['platform_type'] == 'Float']

In [12]:
dates_of_gliders = positions_from_start[positions_from_start['platform_type'].isin(['glider', 'Float'])]['date'].unique()

In [15]:
current = pd.read_csv(os.path.join(data_dir, 'Gliders/current.csv'))
current['datetime'] = current['timestamp'].apply(lambda x: datetime.utcfromtimestamp(x / 1000))
current_position = current[current['variable'].isin(['m_lon', 'm_lat'])]
current_x = current[current['variable'] == 'm_water_vx']
current_y = current[current['variable'] == 'm_water_vy']
current_position = current_position.pivot_table(index='datetime', columns='variable', values='value').reset_index()
current_x =  current_x.pivot_table(index='datetime', columns='variable', values='value').reset_index()
current_y =  current_y.pivot_table(index='datetime', columns='variable', values='value').reset_index()


# Ensure both DataFrames are sorted by the datetime column
current_position = current_position.sort_values('datetime')
current_x = current_x.sort_values('datetime')
current_y = current_y.sort_values('datetime')


# Perform the asof merge
merged_df = pd.merge_asof(current_x, current_y, on='datetime', direction='nearest')
merged_df = pd.merge_asof(merged_df, current_position, on='datetime', direction='nearest')

C:\Users\flapet\AppData\Local\Temp\ipykernel_10848\662352737.py:2: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  current['datetime'] = current['timestamp'].apply(lambda x: datetime.utcfromtimestamp(x / 1000))


In [17]:
for i in tqdm(dates_of_gliders):
    temp = all_positions[all_positions['datetime']<= pd.to_datetime(i)]
    temp = temp.loc[temp.groupby('platform_id')['datetime'].idxmax()]
    datename = i.replace(" ", "_").replace(":", "").replace("-", "")
    filename = parent_dir + '/Output/Plots/rt_tracking/rt_tracking_' + datename + '.png'

    temp_current = merged_df[merged_df['datetime']<= pd.to_datetime(i)]
    x = temp_current['m_lon']
    y = temp_current['m_lat']
    u = temp_current['m_water_vx']
    v = temp_current['m_water_vy']

    speed = np.sqrt(u**2 + v**2)


    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mercator())

                        # Set the map extent based on your latitude and longitude ranges
    ax.set_extent([min_lon, max_lon, min_lat, max_lat], crs=ccrs.PlateCarree())


    for platform_type, mask in platform_mask.items():
        subset = temp[temp['platform_type'] == platform_type]
        for platform_id, color in platform_colors.items():
            sub_subset = subset[subset['platform_id'] == platform_id]
            if not sub_subset.empty:
                ax.scatter(sub_subset['lon'], sub_subset['lat'], c=color, label=platform_id, marker=mask, s=100, transform=ccrs.PlateCarree())
                for index, row in sub_subset.iterrows():
                    ax.annotate(str(row['date']), (row['lon'], row['lat']), transform=ccrs.PlateCarree())

    ax.scatter(-24, 60, label = 'Station 1', marker = 'X', c = 'Black', transform = ccrs.PlateCarree(), s = 200)
    im = ax.quiver(x, y, u, v, speed, angles='xy', scale_units='xy', cmap='viridis', transform=ccrs.PlateCarree(), width = 0.002)


    # Add a scale bar
    ax.add_artist(ScaleBar(1, location = "lower left"))

    # Add gridlines and labels
    gl = ax.gridlines(draw_labels=True)
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 10}
    gl.ylabel_style = {'size': 10}
    gl.top_labels=False   # suppress top labels
    gl.right_labels=False # suppress right labels

    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title(f'Platform Positions \n {i}')
    # Shink current axis by 20%
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    plt.legend(title='Platform ID', bbox_to_anchor=(1.05, 1), borderaxespad=0., loc='upper left')  # Adjust the coordinates as needed

    plt.savefig(filename)
    plt.close()


100%|██████████| 10/10 [00:08<00:00,  1.12it/s]


In [ ]:
    """
current = pd.read_csv(os.path.join(data_dir, 'Gliders/current.csv'))
current['datetime'] = current['timestamp'].apply(lambda x: datetime.utcfromtimestamp(x / 1000))
current_position = current[current['variable'].isin(['m_lon', 'm_lat'])]
current_x = current[current['variable'] == 'm_water_vx']
current_y = current[current['variable'] == 'm_water_vy']
current_position = current_position.pivot_table(index='datetime', columns='variable', values='value').reset_index()
current_x =  current_x.pivot_table(index='datetime', columns='variable', values='value').reset_index()
current_y =  current_y.pivot_table(index='datetime', columns='variable', values='value').reset_index()


# Ensure both DataFrames are sorted by the datetime column
current_position = current_position.sort_values('datetime')
current_x = current_x.sort_values('datetime')
current_y = current_y.sort_values('datetime')


# Perform the asof merge
merged_df = pd.merge_asof(current_x, current_y, on='datetime', direction='nearest')
merged_df = pd.merge_asof(merged_df, current_position, on='datetime', direction='nearest')

x = merged_df['m_lon']
y = merged_df['m_lat']
u = merged_df['m_water_vx']
v = merged_df['m_water_vy']

speed = np.sqrt(u**2 + v**2)
    x = merged_df['m_lon']
y = merged_df['m_lat']
u = merged_df['m_water_vx']
v = merged_df['m_water_vy']

speed = np.sqrt(u**2 + v**2)

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mercator())
ax.set_extent([min_lon, max_lon, min_lat, max_lat], crs=ccrs.PlateCarree())

im = ax.quiver(x, y, u, v, speed, angles='xy', scale_units='xy', cmap='viridis', transform=ccrs.PlateCarree(), regrid_shape=50, width = 0.002)
plt.show()
    """